In [1]:
# import os
# vipshome = r'C:\Users\Kevin\Downloads\vips-dev-8.14\bin'
# os.environ['PATH'] = vipshome + ';' + os.environ['PATH']
# import pyvips

In [14]:
# import os
# add_dll_dir = getattr(os, "add_dll_directory", None)
# vipsbin = r'C:\Users\Kevin\Downloads\vips-dev-8.14\bin'  # LibVIPS binary dir
# if callable(add_dll_dir):
#     add_dll_dir(vipsbin)
# else:
#     os.environ["PATH"] = os.pathsep.join((vipsbin, os.environ["PATH"]))
#
# import pyvips

OSError: cannot load library 'C:\Users\Kevin\Downloads\vips-dev-8.14\bin\libvips-42.dll': error 0x7f

In [1]:
# #importing openslide:
# OPENSLIDE_PATH = r'C:\Users\Kevin\Downloads\openslide-win64-20221217\bin'
#
# import os
#
# if hasattr(os, 'add_dll_directory'):
#     # Python >= 3.8 on Windows
#     with os.add_dll_directory(OPENSLIDE_PATH):
#         import openslide
# else:
#     import openslide

In [9]:
import pyvips
from PIL import Image
import numpy as np
import os
from time import time
from natsort import natsorted
import pandas as pd
import time


def svs2tiff(svs,rsf,svs_dst):
    start = time.time()
    if not os.path.exists(svs_dst): os.mkdir(svs_dst)
    src,fn = os.path.split(svs)
    fn,ext = os.path.splitext(fn)
    fn1 = fn + '.ome.tiff'
    if os.path.exists(os.path.join(svs_dst,fn1)): return
    print(fn)

    svs_obj = openslide.OpenSlide(svs)
    svs_img = svs_obj.read_region(location=(0,0),level=0,size=svs_obj.level_dimensions[0]).convert('RGB')
        # openslide will add an alpha ... drop it

    resize_factorx = rsf/float(svs_obj.properties['openslide.mpp-x']) #8um = 1.25x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = rsf/float(svs_obj.properties['openslide.mpp-y'])

    resize_dimension = tuple([int(np.ceil(svs_obj.dimensions[0]/resize_factorx)),int(np.ceil(svs_obj.dimensions[1]/resize_factory))])
    svs_img = svs_img.resize(resize_dimension,resample=Image.NEAREST)


    # pyvips
    svs_img = pyvips.Image.new_from_array(obj = svs_img)
    if svs_img.hasalpha():
        svs_img = svs_img[:-1]
    image_height = svs_img.height
    image_bands = svs_img.bands

    # split to separate image planes and stack vertically ready for OME
    svs_img = pyvips.Image.arrayjoin(svs_img.bandsplit(), across=1)

    # set minimal OME metadata
    # before we can modify an image (set metadata in this case), we must take a
    # private copy
    svs_img = svs_img.copy()
    svs_img.set_type(pyvips.GValue.gint_type, "page-height", image_height)
    svs_img.set_type(pyvips.GValue.gstr_type, "image-description",
                f"""<?xml version="1.0" encoding="UTF-8"?>
    <OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
        xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
        <Image ID="Image:0">
            <!-- Minimum required fields about image dimensions -->
            <Pixels DimensionOrder="XYCZT"
                    ID="Pixels:0"
                    SizeC="{image_bands}"
                    SizeT="1"
                    SizeX="{im.width}"
                    SizeY="{image_height}"
                    SizeZ="1"
                    Type="uint8">
            </Pixels>
        </Image>
    </OME>""")

    svs_img.tiffsave(os.path.join(svs_dst,fn1), compression="jpeg", tile=True,
                tile_width=512, tile_height=512,
                pyramid=True, subifd=True)
    print("Image sucessfully saved!")
    end = time.time()
    print("Time it took to generate one ometiff from one ndpi: {} minutes".format(round((end-start)/60)))

OSError: cannot load library 'libvips-42.dll': error 0x7e.  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libvips-42.dll'

In [ ]:
import os
from tqdm import tqdm
# Define the input and output directories
input_dir = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE'
output_dir = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um'

filenames = [x for x in os.listdir(input_dir) if x.endswith(".ndpi")]
# Loop through all the files in the input directory
for filename in tqdm(filenames,total=len(filenames),desc='Processed ndpis:',colour='red')
    input_filepath = os.path.join(input_dir, filename)
    output_filename = filename.split(".")[0] + ".ome.tiff"
    output_filepath = os.path.join(output_dir, output_filename)
    svs2tiff(input_filepath, 1, output_filepath)
